## IMPORTS

In [2]:
import os
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

## LOAD DATASET AND GET TOKENS, SLOTS AND INTENTS, AND THEIR INDEXES

In [3]:
def load_ds(fname):
    fname = os.path.join(fname)
    with open(fname, 'rb') as stream:
        ds, dicts = pickle.load(stream)
    print('Done  loading: ', fname)
    print('      samples: {:4d}'.format(len(ds['query'])))
    print('   vocab_size: {:4d}'.format(len(dicts['token_ids'])))
    print('   slot count: {:4d}'.format(len(dicts['slot_ids'])))
    print(' intent count: {:4d}'.format(len(dicts['intent_ids'])))
    return ds, dicts

In [4]:
train_ds, dicts = load_ds('atis.train.pkl')
test_ds, _ = load_ds('atis.test.pkl')

Done  loading:  atis.train.pkl
      samples: 4978
   vocab_size:  943
   slot count:  129
 intent count:   26
Done  loading:  atis.test.pkl
      samples:  893
   vocab_size:  943
   slot count:  129
 intent count:   26


In [5]:
t2i, s2i, in2i = map(dicts.get, ['token_ids', 'slot_ids', 'intent_ids'])
i2t, i2s, i2in = map(lambda d: {d[k]: k for k in d.keys()}, [t2i, s2i, in2i])
query, slots, intent = map(train_ds.get,
                           ['query', 'slot_labels', 'intent_labels'])

In [6]:
t2i_test, s2i_test, in2i_test = map(dicts.get, ['token_ids', 'slot_ids', 'intent_ids'])
i2t_test, i2s_test, i2in_test = map(lambda d: {d[k]: k for k in d.keys()}, [t2i_test, s2i_test, in2i_test])
query_test, slots_test, intent_test = map(test_ds.get,
                           ['query', 'slot_labels', 'intent_labels'])

## FORMAT TRAIN AND TEST DATASETS

In [7]:
query_complete = query + query_test
intent_complete = intent + intent_test # TURN INTO A SINGLE DATASET
padded_x = pad_sequences(query_complete) # PAD QUERIES FOR UNIFORMIZATION
y = tf.keras.utils.to_categorical(intent_complete) # TURN Y FROM CATEGORICAL TO ONE_HOT_ENCODING

In [8]:
X_train, X_test, y_train, y_test = train_test_split(padded_x, y, test_size=0.2, random_state=42)

## HYPERPARAMETERS

In [9]:
INPUT_LENGTH = 48
EMBEDDING_SIZE = 100
UNITS = 128
DROPOUT = 0.2
### Training parameters
LOSS = "categorical_crossentropy"
OPTIMIZER = "adam"
BATCH_SIZE = 32
EPOCHS = 8

## MODEL CREATION AND COMPILING

In [10]:
model = Sequential()
model.add(Embedding(len(dicts['token_ids'])+1, EMBEDDING_SIZE, input_length=INPUT_LENGTH))
model.add(LSTM(UNITS, return_sequences=True)) #p/ mais de uma camada
model.add(Dropout(DROPOUT))
model.add(LSTM(UNITS,return_sequences=False))
model.add(Dropout(DROPOUT))
model.add(Dense(26, activation='softmax'))

In [11]:
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 48, 100)           94400     
_________________________________________________________________
lstm (LSTM)                  (None, 48, 128)           117248    
_________________________________________________________________
dropout (Dropout)            (None, 48, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 26)                3354      
Total params: 346,586
Trainable params: 346,586
Non-trainable params: 0
__________________________________________________

## MODEL FITTING

In [12]:
history = model.fit(X_train, y_train, batch_size = BATCH_SIZE, epochs=EPOCHS,validation_split=0.15,verbose=2)

Epoch 1/2
125/125 - 29s - loss: 1.3505 - accuracy: 0.7191 - val_loss: 1.0969 - val_accuracy: 0.7518
Epoch 2/2
125/125 - 23s - loss: 1.2151 - accuracy: 0.7256 - val_loss: 1.0196 - val_accuracy: 0.7518


## MODEL TESTING

In [13]:
score = model.evaluate(X_test, y_test, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

37/37 [==============================] - 3s 67ms/step - loss: 1.0753 - accuracy: 0.7421
Test loss: 1.0753177404403687 / Test accuracy: 0.7421276569366455
